In [14]:
import pandas as pd
import datetime as dt
import yfinance as yf
import json
from stockstats import wrap 
import os



# get watch_list

In [55]:
with open('watch_list.json') as f:
    watch_list = json.load(f)

# fetch data from yfinance api

In [71]:
#create stock_data directory if it doesn't exist
if not os.path.exists('stock_data'):
    os.makedirs('stock_data')
for symbol in watch_list:
    tk = yf.Ticker(f"{symbol}.BK")
    df = tk.history(period='2y', interval='1d')
    df.to_csv(f"stock_data/{symbol}.csv")




# apply indicator to data

In [ ]:
#create stock_data_with_indicator directory if it doesn't exist
if not os.path.exists('stock_data_with_indicator'):
    os.makedirs('stock_data_with_indicator')
for symbol in watch_list:
    df = pd.read_csv(f"stock_data\{symbol}.csv")
    df_ta = wrap(df)
    df_ta.init_all()
    df_ta.to_csv(f"stock_data_with_indicator\{symbol}.csv")
df_ta["close_20_ema"]

# show result with plotly and save result to file

In [ ]:
import random
import plotly.graph_objects as go
from tradingview_ta import TA_Handler, Interval

symbol = random.choice(watch_list)
handler = TA_Handler(
    symbol=symbol,
    exchange="SET",
    screener="thailand",
    interval=Interval.INTERVAL_1_DAY,
    timeout=None
)
df = pd.read_csv(f"stock_data_with_indicator\{symbol}.csv")
df.set_index('date', inplace=True)
fig = go.Figure(data=[go.Candlestick(x=df.index,
                                     open=df['open'],
                                     high=df['high'],
                                     low=df['low'],
                                     close=df['close'],
                                     hoverlabel=dict(bgcolor='white'),
                                     name=symbol
                                        )])


# fig.add_trace(go.Scatter(
#     x=df.index, y=df["middle"], line_shape='spline', name="Middle",))
fig.add_trace(go.Scatter(
    x=df.index, y=df["close_10_sma"], line_shape='spline', name="close_10_sma",))
fig.add_trace(go.Scatter(
    x=df.index, y=df["close_50_sma"], line_shape='spline', name="close_50_sma",))

# fig name
fig.update_layout(
    title=f"#{symbol} [last data {pd.to_datetime(df.index)[-1].strftime('y%Y/ m%d/ d%m')}]<br>{' || '.join([f'{k} : {v}' for k,v in handler.get_analysis().summary.items()])}<br>[ open = {df['open'].iloc[-1]:.2f} || high = {df['high'].iloc[-1]:.2f} || low = {df['low'].iloc[-1]:.2f} || close = {df['close'].iloc[-1]:.2f} ]")
# hide bottom range slider
fig.update_layout(xaxis_rangeslider_visible=False)

# removing all empty dates
dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

if not os.path.exists("images"):
    os.mkdir("images")

fig.write_image(f"images/{symbol}-{dt.date.today()}.webp",format="webp",scale=4)
fig.show()


In [ ]:
from plot import plot_supertrend
import pandas as pd
import yfinance as yf
import json
from stockstats import wrap
import os

# setting
WATCH_LIST_FILE = 'watch_list.json'
STOCK_DATA_DIR = 'stock_data'
STOCK_DATA_DIR_WITH_INDICATOR_DIR = 'stock_data_with_indicator'


# get watch list
file_path = os.path.dirname(__file__)
with open(os.path.join(file_path, WATCH_LIST_FILE)) as f:
    watch_list = json.load(f)

# create stock data dir if not exist
if not os.path.exists(os.path.join(file_path, STOCK_DATA_DIR)):
    os.makedirs(os.path.join(file_path, STOCK_DATA_DIR))

# fetch stock data from yahoo finance
for i, symbol in enumerate(watch_list):
    tk = yf.Ticker(f"{symbol}.BK")
    df = tk.history(period='3mo', interval='1d')
    df.to_csv(os.path.join(file_path, STOCK_DATA_DIR, f"{symbol}.csv"))
    print(f"{symbol} data fetched {i+1}/{len(watch_list)}")

# create stock data with indicator dir if not exist
if not os.path.exists(os.path.join(file_path, STOCK_DATA_DIR_WITH_INDICATOR_DIR)):
    os.makedirs(os.path.join(file_path, STOCK_DATA_DIR_WITH_INDICATOR_DIR))

# apply stock stats to stock data
for i, symbol in enumerate(watch_list):
    try:
        df = pd.read_csv(os.path.join(
            file_path, STOCK_DATA_DIR, f"{symbol}.csv"))

        df_ta = wrap(df)
        df_ta["close_20_ema"]
        df_ta["supertrend"]
        df_ta.to_csv(os.path.join(
            file_path, STOCK_DATA_DIR_WITH_INDICATOR_DIR, f"{symbol}.csv"))
        print(f"{symbol} done {i+1}/{len(watch_list)}")
    except Exception as e:
        print(f"{symbol} error {e}")
        continue


In [3]:
tk = yf.Ticker(f"^SET.BK")
info = tk.info
info

{'exchange': 'SET',
 'shortName': 'SET Index',
 'exchangeTimezoneName': 'Asia/Bangkok',
 'exchangeTimezoneShortName': 'ICT',
 'isEsgPopulated': False,
 'gmtOffSetMilliseconds': '25200000',
 'quoteType': 'INDEX',
 'symbol': '^SET.BK',
 'messageBoardId': 'finmb_INDEXSET.BK',
 'market': 'th_market',
 'previousClose': 1536.82,
 'regularMarketOpen': 1538.76,
 'twoHundredDayAverage': 1626.6699,
 'trailingAnnualDividendYield': None,
 'payoutRatio': None,
 'volume24Hr': None,
 'regularMarketDayHigh': 1539.96,
 'navPrice': None,
 'averageDailyVolume10Day': 7071210,
 'totalAssets': None,
 'regularMarketPreviousClose': 1536.82,
 'fiftyDayAverage': 1673.8798,
 'trailingAnnualDividendRate': None,
 'open': 1538.76,
 'toCurrency': None,
 'averageVolume10days': 7071210,
 'expireDate': None,
 'yield': None,
 'algorithm': None,
 'dividendRate': None,
 'exDividendDate': None,
 'beta': None,
 'circulatingSupply': None,
 'startDate': None,
 'regularMarketDayLow': 1517.51,
 'priceHint': 2,
 'currency': 'THB

In [15]:
from plotly import __version__

import cufflinks as cf

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode (connected=True)

cf.go_offline()

In [16]:
cf.get_config_file()

{'sharing': 'public',
 'theme': 'pearl',
 'colorscale': 'dflt',
 'offline': True,
 'offline_connected': True,
 'offline_url': '',
 'offline_show_link': True,
 'offline_link_text': 'Export to plot.ly',
 'datagen_mode': 'stocks',
 'dimensions': None,
 'margin': None,
 'offline_config': None}

In [5]:
cf.set_config_file(theme='pearl',sharing='public',offline=True)

In [6]:
df = pd.read_csv('stock_data\ACE.csv', index_col=0, parse_dates=True)

In [61]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-07-15,3.875000,4.014103,3.855128,3.914744,221314700,0.0,0
2021-07-16,4.073718,4.133333,3.894872,3.974359,407186800,0.0,0
2021-07-19,3.835256,3.934616,3.835256,3.894872,70971000,0.0,0
2021-07-20,3.894872,3.954487,3.835256,3.894872,93454200,0.0,0
2021-07-21,3.914744,4.073718,3.894872,4.033974,186034800,0.0,0
...,...,...,...,...,...,...,...
2022-07-08,2.720000,2.720000,2.660000,2.680000,10563400,0.0,0
2022-07-11,2.680000,2.760000,2.660000,2.720000,31432700,0.0,0
2022-07-12,2.700000,2.720000,2.660000,2.680000,6063700,0.0,0


In [19]:
qf=cf.QuantFig(df,title='dfdfdfdfdfdf',legend='right',name='GS',kind='candlestick',theme='solar',)


qf.add_macd(fast_period=12,slow_period=26,signal_period=9,name='macd')
qf.add_ema(periods=200,name='200ema',color='orange')


fig = qf.figure(
)
dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

fig.write_image("tesddt.webp",format='webp',scale=4,width=1600,height=900)
# fig.write_html("tesddt.html")
fig.show()


KeyError: 0